In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

#Load chunk and index the content of html page

loader = WebBaseLoader(web_path=("https://finviz.com/quote.ashx?t=AAPL&p=d",),
                       bs_kwargs=dict(parse_only = bs4.SoupStrainer(
                           class_=("tab-link-news")
                       )))
text_docs = loader.load()

In [ ]:
text_docs

In [ ]:
##Reading the pdf file

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('2402.06196v2.pdf')
docs = loader.load()



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000,chunk_overlap = 200)
documents = text_splitter.split_documents(docs)

In [ ]:
## Vector EMbedding and vector store using OPENAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents[:20],OpenAIEmbeddings())

## USe that free LLM model for this

In [ ]:
## Vector EMbedding and vector store using OLLAMA 
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents[:20],OllamaEmbeddings())

In [ ]:
## USig  FAISS as a vector store

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents[:20],OllamaEmbeddings())

In [ ]:
query = "Give me the paragraph headings"
result = db.similarity_search(query)
result[0].page_content

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama2")
llm

In [ ]:
#Designing ChatPrompt Template

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
    Answer the following question based only on the provided context. 
    Think step by step before providing a detailed answer.
    I will tip you $1000 if the user finds the answer helpful.
    <context>
    {context}
    </context>
    Question: {input}""")

In [ ]:
#Chain Implementation

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
#Retriever
retriever= db.as_retriever()

In [ ]:
#Retrieval Chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "Markov chain models known as the n-gram models"})

In [ ]:
response['answer']